In [6]:
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from config import username, password, api_key
import requests
import datetime
import json
import numpy as np
import requests 
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import io


### CREATING DATABASE

In [10]:
engine = create_engine('postgresql://{}:{}@localhost:5432/cali_renewable_db'.format(username,password))
con = engine.connect()


with engine.connect() as c:
    sql = '''
CREATE SCHEMA "Production";

CREATE SCHEMA "Demand";

CREATE SCHEMA "Comparison";


CREATE TABLE "Production".hourlyrenewable(
"timestamp" timestamp PRIMARY KEY,
"date" date not null,
"Hour" int not null,
"TOTAL" int,
"SOLAR TOTAL" int,
"SOLAR PV" int,
"SOLAR THERMAL" int,
"WIND TOTAL" int,
"GEOTHERMAL" int,
"BIOMASS" int,
"BIOGAS" int,
"SMALL HYDRO" int
);


CREATE TABLE "Production".hourlytotal(
"timestamp" timestamp PRIMARY KEY,
"date" date not null,
"Hour" int not null,
"RENEWABLES" int,
"NUCLEAR" int,
"THERMAL" int,
"HYDRO" int
);


CREATE TABLE "Demand".hourlydemand(
"timestamp" timestamp PRIMARY KEY,
"date" date not null,
"Hour" int not null,
    "DEMAND" int
);


CREATE TABLE "Comparison".percentproductionrenewable(
"Date" timestamp primary key,
"Total Production" float,
"Renewable Production" float,
"Percent Production Renew" float
);


CREATE TABLE "Comparison".percentdemandrenewable(
"Date" timestamp primary key,
"Total Demand" float,
"Renewable Production" float,
"Percent Demand Renew" float
);
    '''
    c.execute(sql)


### Scraping Hourly Production Data from CAISO

In [12]:
today = datetime.datetime.today().strftime('%Y-%m-%d')
dates = np.array(pd.date_range(start='06/01/2015',end=today).date)

data_frames_renewable_prod = {}
data_frames_all_prod = {}

scrape_log = pd.DataFrame(columns=['date','exists','got data'])

for i in dates:
    date = str(i).replace('-','')
    if date in ['20170312','20160313']:
        continue
    
    url = 'http://content.caiso.com/green/renewrpt/{}_DailyRenewablesWatch.txt'.format(date)
    with requests.get(url, stream=True) as r:
        data = bs(r.text, 'html.parser')

        text_data = data.text
        if '404 Not Found' in text_data:
            entry = pd.DataFrame({
                'date':[date],
                'exists':False,
                'got data':False
            })
            scrape_log = pd.concat([scrape_log, entry])

            continue 
        try:
            f = io.StringIO(text_data)
            top = pd.read_csv(f,delimiter='\t+', header=1, skipfooter=26)
            top['date'] = date

            f = io.StringIO(text_data)
            bottom = pd.read_csv(f,delimiter='\t+', skiprows=26, header=1)
            bottom['date'] = date

            data_frames_renewable_prod[date] = top
            data_frames_all_prod[date] = bottom
            entry = pd.DataFrame({
                'date':[date],
                'exists':True,
                'got data':True
            })
            scrape_log = pd.concat([scrape_log, entry])
        except:
            entry = pd.DataFrame({
                'date':[date],
                'exists':True,
                'got data':False
            })
            scrape_log = pd.concat([scrape_log, entry])


        
        
scrape_log[scrape_log.exists == False]

C:\Users\graha\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
C:\Users\graha\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


,date,exists,got data
0,20190923,False,False
0,20190924,False,False
0,20190925,False,False
0,20190926,False,False
0,20190927,False,False
0,20190928,False,False
0,20190929,False,False
0,20190930,False,False
0,20191001,False,False
0,20191002,False,False


### Concat daily scraped tables into two total tables

In [13]:
hourly_renewable_prod = pd.DataFrame([])
hourly_total_prod = pd.DataFrame([])
for i in dates:
    date = str(i).replace('-','')
    try:
        datar = data_frames_renewable_prod[date]
        datat = data_frames_all_prod[date]
    except KeyError:
        continue
    hourly_renewable_prod = pd.concat([hourly_renewable_prod, datar])
    hourly_total_prod = pd.concat([hourly_total_prod, datat])

In [14]:
pd.set_option('display.max_rows', 500)
hourly_renewable_prod['date'] = pd.to_datetime(hourly_renewable_prod['date'])
hourly_total_prod['date'] = pd.to_datetime(hourly_total_prod['date'])
print(hourly_renewable_prod.info())
print(hourly_total_prod.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38160 entries, 0 to 23
Data columns (total 9 columns):
Hour             38160 non-null int64
GEOTHERMAL       38160 non-null int64
BIOMASS          38160 non-null int64
BIOGAS           38160 non-null int64
SMALL HYDRO      38160 non-null int64
WIND TOTAL       38160 non-null int64
SOLAR PV         38160 non-null int64
SOLAR THERMAL    38160 non-null int64
date             38160 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(8)
memory usage: 2.9 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 38160 entries, 0 to 23
Data columns (total 7 columns):
Hour          38160 non-null int64
RENEWABLES    38160 non-null int64
NUCLEAR       38160 non-null int64
THERMAL       38160 non-null int64
IMPORTS       38160 non-null int64
HYDRO         38160 non-null int64
date          38160 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(6)
memory usage: 2.3 MB
None


### Transform

In [15]:
hourly_total_prod['timestamp'] = hourly_total_prod['date'].astype(str) + 'T' + (hourly_total_prod['Hour']-1).astype(str) +':00:00'
hourly_total_prod['timestamp'] = pd.to_datetime(hourly_total_prod['timestamp'])
hourly_total_prod = hourly_total_prod[['timestamp','date','Hour','RENEWABLES','NUCLEAR','THERMAL','HYDRO']]

hourly_renewable_prod['timestamp'] = hourly_renewable_prod['date'].astype(str) + 'T' + (hourly_renewable_prod['Hour']-1).astype(str) +':00:00'
hourly_renewable_prod['timestamp'] = pd.to_datetime(hourly_renewable_prod['timestamp'])
hourly_renewable_prod['SOLAR TOTAL'] = hourly_renewable_prod['SOLAR PV'] + hourly_renewable_prod['SOLAR THERMAL']
hourly_renewable_prod['TOTAL'] = hourly_renewable_prod['SOLAR TOTAL'] + hourly_renewable_prod['WIND TOTAL'] +\
hourly_renewable_prod['GEOTHERMAL'] +hourly_renewable_prod['BIOMASS'] +hourly_renewable_prod['BIOGAS'] +hourly_renewable_prod['SMALL HYDRO']
hourly_renewable_prod = hourly_renewable_prod[['timestamp','date','Hour','TOTAL','SOLAR TOTAL','SOLAR PV','SOLAR THERMAL',
                                               'WIND TOTAL','GEOTHERMAL','BIOMASS','BIOGAS','SMALL HYDRO']]


display(hourly_renewable_prod.head())
display(hourly_total_prod.head())

,timestamp,date,Hour,TOTAL,SOLAR TOTAL,SOLAR PV,SOLAR THERMAL,WIND TOTAL,GEOTHERMAL,BIOMASS,BIOGAS,SMALL HYDRO
0,2015-06-01 00:00:00,2015-06-01,1,4669,0,0,0,2942,1038,321,209,159
1,2015-06-01 01:00:00,2015-06-01,2,4396,0,0,0,2708,1040,316,210,122
2,2015-06-01 02:00:00,2015-06-01,3,4197,0,0,0,2509,1039,319,209,121
3,2015-06-01 03:00:00,2015-06-01,4,4059,0,0,0,2372,1035,320,209,123
4,2015-06-01 04:00:00,2015-06-01,5,3694,0,0,0,1973,1035,317,208,161


,timestamp,date,Hour,RENEWABLES,NUCLEAR,THERMAL,HYDRO
0,2015-06-01 00:00:00,2015-06-01,1,4668,2280,7318,1050
1,2015-06-01 01:00:00,2015-06-01,2,4396,2277,7383,1077
2,2015-06-01 02:00:00,2015-06-01,3,4197,2274,6921,1066
3,2015-06-01 03:00:00,2015-06-01,4,4058,2275,6565,1072
4,2015-06-01 04:00:00,2015-06-01,5,3694,2276,7050,1159


### Load

In [16]:

name_total = 'hourlytotal'
name_renew = 'hourlyrenewable'
schema = 'Production'
hourly_renewable_prod.to_sql(name=name_renew,con=con,schema=schema, if_exists='append',
                            index=False)

In [17]:
hourly_total_prod.to_sql(name=name_total, con=con, schema=schema,
                        if_exists='append', index=False)

### EIA API REQUEST FOR DEMAND DATA

In [18]:
# Use API to get demand dataset
url = f'https://api.eia.gov/series/?api_key={api_key}&series_id=EBA.CISO-ALL.D.HL'
response = requests.get(url)

In [19]:

# Review json for format
response_json = response.json()
# print(response_json)
num_resp=len(response_json['series'][0]['data'])
print(f'{num_resp} responses')
print(response_json['series'][0]['data'][0])
print(response_json['series'][0]['data'][-1])

39371 responses
['20191229T11-08', 21305]
['20150701T01-07', 31486]


In [20]:
# Pull data for dataframe from nested dictionary
dict = []
for response in range(num_resp):
    dict.append(response_json['series'][0]['data'][response])

In [21]:
# Create datafame and check results
df = pd.DataFrame(dict)
print(df.head())
print(df.tail())

                0        1
0  20191229T11-08  21305.0
1  20191229T10-08  21638.0
2  20191229T09-08  21745.0
3  20191229T08-08  21859.0
4  20191229T07-08  21586.0
                    0        1
39366  20150701T05-07  25661.0
39367  20150701T04-07  26388.0
39368  20150701T03-07  27416.0
39369  20150701T02-07  28989.0
39370  20150701T01-07  31486.0


In [22]:
# Rename columns
demand_df = df.rename(columns={0:'timestamp', 1:'DEMAND'})
# Clean up timestamp by removing end string (08,07)
new = demand_df['timestamp'].str.split("-", n = 1, expand = True) 
demand_df['timestamp']= new[0] 
demand_df['timestamp'] = pd.to_datetime(demand_df['timestamp'])

# Splint datetimes as needed:
# demand_df['Year'] = demand_df['Timestamp'].dt.year
# demand_df['Month'] = demand_df['Timestamp'].dt.month
# demand_df['Day'] = demand_df['Timestamp'].dt.day
demand_df['Hour'] = demand_df['timestamp'].dt.hour # Hour 0 = midnight
demand_df['date'] = demand_df['timestamp'].dt.date

# There were duplicate listing for a handful of dates
# Dropped so not to violate primary key rules
demand_df.drop_duplicates(subset ="timestamp", 
                     keep = 'last', inplace = True) 

demand_df.head(24)

,timestamp,DEMAND,Hour,date
0,2019-12-29 11:00:00,21305.0,11,2019-12-29
1,2019-12-29 10:00:00,21638.0,10,2019-12-29
2,2019-12-29 09:00:00,21745.0,9,2019-12-29
3,2019-12-29 08:00:00,21859.0,8,2019-12-29
4,2019-12-29 07:00:00,21586.0,7,2019-12-29
5,2019-12-29 06:00:00,20756.0,6,2019-12-29
6,2019-12-29 05:00:00,20300.0,5,2019-12-29
7,2019-12-29 04:00:00,20260.0,4,2019-12-29
8,2019-12-29 03:00:00,20497.0,3,2019-12-29
9,2019-12-29 02:00:00,21000.0,2,2019-12-29


In [23]:

name_demand = "hourlydemand"
schema = 'Demand'
demand_df.to_sql(name=name_demand,con=con,schema=schema, if_exists='append',
                            index=False)

In [24]:
with engine.connect() as c:
    c.execute('''
alter table "Production".hourlytotal
add column "TOTAL PRODUCTION" int;


update "Production".hourlytotal
set "TOTAL PRODUCTION" = "RENEWABLES"+"NUCLEAR"+"THERMAL"+"HYDRO";




insert into "Comparison".percentproductionrenewable 
select r."timestamp" as "Date",
sum(t."TOTAL PRODUCTION") as "Total Production",
sum(r."TOTAL") as "Renewable Production"
from "Production".hourlyrenewable as r
left join 
"Production".hourlytotal as t
on
r.timestamp = t.timestamp
group by "Date"
order by "Date" desc;

update "Comparison".percentproductionrenewable
set "Percent Production Renew" = "Renewable Production" / "Total Production";



insert into "Comparison".percentdemandrenewable 
select r."timestamp" as "Date",
sum(d."DEMAND") as "Total Demand",
sum(r."TOTAL") as "Renewable Production"
from "Production".hourlyrenewable as r
left join 
"Demand".hourlydemand as d
on
r.timestamp = d.timestamp
group by "Date"
order by "Date" desc;

update "Comparison".percentdemandrenewable
set "Percent Demand Renew" = "Renewable Production" / "Total Demand";

    ''')